<a href="https://colab.research.google.com/github/skasana/fake-news-detection-using-machine-learning/blob/main/Fake_News_Detection_using_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import string
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [18]:
df_fake = pd.read_csv("/content/Fake.csv", on_bad_lines='skip', engine='python')
df_true = pd.read_csv("/content/True.csv", on_bad_lines='skip', engine='python')

In [20]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [21]:
df_true.tail()

,title,text,subject,date
1612,State funding changes in spotlight in Republic...,WASHINGTON (Reuters) - Republican leaders soug...,politicsNews,"September 21, 2017"
1613,Republican Senator Collins 'leaning against' n...,WASHINGTON (Reuters) - Republican Senator Susa...,politicsNews,"September 22, 2017"
1614,Ex-Republican presidential candidate Bob Dole ...,(Reuters) - Former Republican presidential can...,politicsNews,"September 22, 2017"
1615,What's a 'dotard' anyway? Kim's insult to Trump,(Reuters) - North Korean leader Kim Jong Un ha...,politicsNews,"September 22, 2017"
1616,"Trump, South Korea's Moon agree to boost defen...",WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"September 22, 2017"


In [22]:
df_fake["class"]=0
df_true['class']=1

In [23]:
df_fake.shape, df_true.shape

((1994, 5), (1617, 5))

In [25]:
df_fake_manual_testing = df_fake.tail(10)
for i in range(23480, 23470, -1):
    if i in df_fake.index:
        df_fake.drop([i], axis=0, inplace=True)

df_true_manual_testing = df_true.tail(10)
for i in range(21416, 21406, -1):
    if i in df_true.index:
        df_true.drop([i], axis=0, inplace=True)

In [26]:
df_fake.shape, df_true.shape

((1994, 5), (1617, 5))

In [27]:
df_fake_manual_testing["class"] = 0
df_true_manual_testing["class"] = 1

<ipython-input-27-3aaf8ec2aad1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fake_manual_testing["class"] = 0
<ipython-input-27-3aaf8ec2aad1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_true_manual_testing["class"] = 1


In [28]:
df_fake_manual_testing.head(10)

,title,text,subject,date,class
1984,Nunes ADMITS He Was Secretly At WH Right Befo...,If there is one upside to having this level of...,News,"March 27, 2017",0
1985,WATCH: After Facing Angry Constituents At Tow...,Just one day after Paul Ryan failed to garner ...,News,"March 27, 2017",0
1986,Obama’s Press Secretary Just Got A New Job; T...,Donald Trump is soon going to be unleashing ev...,News,"March 27, 2017",0
1987,Paul Ryan Was So Desperate To Pass Failed Tru...,This is what being pathetic looks like.House S...,News,"March 27, 2017",0
1988,Dan Rather: Trump’s First Two Months In Offic...,Legendary newsman Dan Rather has lived through...,News,"March 27, 2017",0
1989,BREAKING: Devin Nunes Is Now The Subject Of A...,"Embattled House Intel Chairman, Devin Nunes (R...",News,"March 26, 2017",0
1990,Fox News Frantically Tries To Cover Up Trump’...,It s not clear whether Trump called in a favor...,News,"March 26, 2017",0
1991,California Governor Delivers HARSH Message To...,It s no secret that Californians aren t really...,News,"March 26, 2017",0
1992,The Internet Goes For Newt Gingrich’s Throat ...,Donald Trump repeatedly attacked former Presid...,News,"March 26, 2017",0
1993,"Trump Insulted German Leader On First Visit, ...",The fact that German Chancellor Angela Merkel ...,News,"March 26, 2017",0


In [29]:
df_true_manual_testing.head(10)

,title,text,subject,date,class
1607,Trump yet to decide on new travel restrictions...,WASHINGTON (Reuters) - The acting secretary of...,politicsNews,"September 22, 2017",1
1608,Senator McCain to vote against Obamacare repea...,WASHINGTON (Reuters) - U.S. Senator John McCai...,politicsNews,"September 22, 2017",1
1609,Health Secretary Price's use of private jets u...,WASHINGTON (Reuters) - U.S. Health Secretary T...,politicsNews,"September 22, 2017",1
1610,U.S. officials to hold Friday briefing on 'enh...,WASHINGTON (Reuters) - Officials from the Depa...,politicsNews,"September 22, 2017",1
1611,Factbox: Trump on Twitter (Sept 22) - North Ko...,The following statements were posted to the ve...,politicsNews,"September 22, 2017",1
1612,State funding changes in spotlight in Republic...,WASHINGTON (Reuters) - Republican leaders soug...,politicsNews,"September 21, 2017",1
1613,Republican Senator Collins 'leaning against' n...,WASHINGTON (Reuters) - Republican Senator Susa...,politicsNews,"September 22, 2017",1
1614,Ex-Republican presidential candidate Bob Dole ...,(Reuters) - Former Republican presidential can...,politicsNews,"September 22, 2017",1
1615,What's a 'dotard' anyway? Kim's insult to Trump,(Reuters) - North Korean leader Kim Jong Un ha...,politicsNews,"September 22, 2017",1
1616,"Trump, South Korea's Moon agree to boost defen...",WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"September 22, 2017",1


In [30]:
df_manual_testing = pd.concat([df_fake_manual_testing,df_true_manual_testing], axis = 0)
df_manual_testing.to_csv("manual_testing.csv")

In [31]:
#merge true and fake dataframes
df_merge = pd.concat([df_fake, df_true], axis =0 )
df_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [32]:
df_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [33]:
#removing columns which are not required
df = df_merge.drop(["title", "subject","date"], axis = 1)

In [34]:
df.isnull().sum()

,0
text,0
class,0


In [35]:
#random shuffling the dataframe
df = df.sample(frac = 1)

In [36]:
df.head()

,text,class
1314,WASHINGTON (Reuters) - Although U.S. President...,1
1105,WASHINGTON (Reuters) - U.S. President Donald T...,1
708,WASHINGTON (Reuters) - U.S. Senate Republicans...,1
668,"First, he impaneled a grand jury. Then the gra...",0
1248,One of the major things with Donald Trump s sp...,0


In [37]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [38]:
df.columns

Index(['text', 'class'], dtype='object')

In [39]:
df.head()

,text,class
0,WASHINGTON (Reuters) - Although U.S. President...,1
1,WASHINGTON (Reuters) - U.S. President Donald T...,1
2,WASHINGTON (Reuters) - U.S. Senate Republicans...,1
3,"First, he impaneled a grand jury. Then the gra...",0
4,One of the major things with Donald Trump s sp...,0


In [40]:
#creating a function to process the texts
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [41]:
df["text"] = df["text"].apply(wordopt)

In [42]:
#defining dependent and independent variables
x = df["text"]
y = df["class"]

In [43]:
#splitting training and testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [44]:
#convert text to vectors
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [46]:
#logistic regression
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [49]:
pred_lr=LR.predict(xv_test)
LR.score(xv_test, y_test)

0.9922480620155039

In [50]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       470
           1       1.00      0.98      0.99       433

    accuracy                           0.99       903
   macro avg       0.99      0.99      0.99       903
weighted avg       0.99      0.99      0.99       903



In [51]:
#decision tree classification
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [52]:
pred_dt = DT.predict(xv_test)

In [53]:
DT.score(xv_test, y_test)

0.9977851605758582

In [54]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       470
           1       1.00      1.00      1.00       433

    accuracy                           1.00       903
   macro avg       1.00      1.00      1.00       903
weighted avg       1.00      1.00      1.00       903



In [55]:
#gradient boosting classifier
from sklearn.ensemble import GradientBoostingClassifier
GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [56]:
pred_gbc = GBC.predict(xv_test)

In [57]:
GBC.score(xv_test, y_test)

0.9988925802879292

In [58]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       470
           1       1.00      1.00      1.00       433

    accuracy                           1.00       903
   macro avg       1.00      1.00      1.00       903
weighted avg       1.00      1.00      1.00       903



In [59]:
#random forest classifier
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [60]:
pred_rfc = RFC.predict(xv_test)

In [61]:
RFC.score(xv_test, y_test)

0.9977851605758582

In [62]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       470
           1       1.00      1.00      1.00       433

    accuracy                           1.00       903
   macro avg       1.00      1.00      1.00       903
weighted avg       1.00      1.00      1.00       903



In [63]:
#model training
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"

def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]),                                                                                                       output_lable(pred_DT[0]),
                                                                                                              output_lable(pred_GBC[0]),
                                                                                                              output_lable(pred_RFC[0])))

In [64]:
news = str(input())
manual_testing(news)

BRUSSELS (Reuters) - NATO allies on Tuesday welcomed President Donald Trump s decision to commit more forces to Afghanistan, as part of a new U.S. strategy he said would require more troops and funding from America s partners. Having run for the White House last year on a pledge to withdraw swiftly from Afghanistan, Trump reversed course on Monday and promised a stepped-up military campaign against  Taliban insurgents, saying:  Our troops will fight to win .  U.S. officials said he had signed off on plans to send about 4,000 more U.S. troops to add to the roughly 8,400 now deployed in Afghanistan. But his speech did not define benchmarks for successfully ending the war that began with the U.S.-led invasion of Afghanistan in 2001, and which he acknowledged had required an   extraordinary sacrifice of blood and treasure .  We will ask our NATO allies and global partners to support our new strategy, with additional troops and funding increases in line with our own. We are confident they w

In [65]:
news = str(input())
manual_testing(news)

Vic Bishop Waking TimesOur reality is carefully constructed by powerful corporate, political and special interest sources in order to covertly sway public opinion. Blatant lies are often televised regarding terrorism, food, war, health, etc. They are fashioned to sway public opinion and condition viewers to accept what have become destructive societal norms.The practice of manipulating and controlling public opinion with distorted media messages has become so common that there is a whole industry formed around this. The entire role of this brainwashing industry is to figure out how to spin information to journalists, similar to the lobbying of government. It is never really clear just how much truth the journalists receive because the news industry has become complacent. The messages that it presents are shaped by corporate powers who often spend millions on advertising with the six conglomerates that own 90% of the media:General Electric (GE), News-Corp, Disney, Viacom, Time Warner, a